In [1]:
import requests


In [5]:
urlcook = 'https://api.pushshift.io/reddit/search/submission/?subreddit=cooking&metadata=true&size=500&is_self=true'
urlbegin = 'https://api.pushshift.io/reddit/search/submission?subreddit=cookingforbeginners&metadata=true&size=500&is_self=true'

In [6]:
rescook = requests.get(urlcook)
resbegin = requests.get(urlbegin)

In [10]:
begindict = resbegin.json()
cookdict= rescook.json()

In [12]:
begindict.keys()

dict_keys(['data', 'metadata'])

In [20]:

begindict.get('data')[0].get('selftext')

'What is the best way to do this? for instance on a griddle, or a pan. I bought an Ozeri Profesional, it says to use a spoon of oil, but how to spread this? I was going to use spray, but it says on the instructions to not use spray.\n\nWith a piece of paper, it is a bit nasty since it can leave paper particles.\n\nis there some PTFE free brush or some kind of tool that will not melt or leave traces?'